In [ ]:
!curl --request GET --url host.docker.internal:21434/api/tags

In [ ]:
from langchain_ollama import ChatOllama

### llm 
local_llm="deepseek-r1:1.5b-qwen-distill-fp16"
llm = ChatOllama(model=local_llm, temperature=0, base_url="http://host.docker.internal:21434")
llm_json = ChatOllama(model=local_llm, temperature=0, return_json=True, base_url="http://host.docker.internal:21434")

In [ ]:
msg = llm.invoke("What is the capital of France?")
msg.content

__________________

# GROQ test

In [ ]:
import os

from groq import Groq
from dotenv import load_dotenv
load_dotenv("../../groq.env")


In [ ]:
client = Groq(
    # This is the default and can be omitted
    api_key=os.getenv("GROQ_API_KEY"),
)

In [ ]:
model_name = os.getenv("GROQ_LLM")
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model=model_name,
    reasoning_format={"type": "groq"},
    temperature=0.6,
    max_completion_tokens=131072,
    top_p=0.95,
    stream=True,
    stop=None,
    
)

In [ ]:

for chunk in chat_completion:
    print(chunk.choices[0].delta.content or "", end="")

__________


# GROQ langchain test

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from ollama_deep_researcher.prompts import (
    query_writer_instructions,
    summarizer_instructions,
    reflection_instructions,
    get_current_date,
)
from dotenv import load_dotenv
load_dotenv("../../groq.env")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model_name = os.getenv("GROQ_LLM")

In [ ]:
current_date = get_current_date()
formatted_prompt = query_writer_instructions.format(
        current_date=current_date, research_topic="fast language models"
    )

In [ ]:
llm = ChatGroq(
    model=model_name,
    temperature=0,
    response_format={"type": "json_object"},
)
#llm_json_mode = llm.with_structured_output(method="json_mode")

In [ ]:
result = llm_json_mode.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [ ]:
result2 = llm.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [ ]:
print(result2.content)

_____

# tavily test (Used for making the crawler search)

In [ ]:
import os
from ollama_deep_researcher.utils import (
    deduplicate_and_format_sources,
    tavily_search,
    format_sources,
)
from dotenv import load_dotenv
load_dotenv("../../.env")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [ ]:
search_results = tavily_search("what is the capital of France?", fetch_full_page=True, max_results=3)

In [ ]:
search_results

In [ ]:
search_str = deduplicate_and_format_sources(
            search_results,
            max_tokens_per_source=10000000,
            fetch_full_page=True,
        )

In [ ]:
print(search_str)

_______

# Test rets_info_scraper and translate

In [ ]:
from ollama_deep_researcher.retsinfo_crawl import retsinfo_search_and_crawl, retsinfo_search, extract_ids, main_crawl, sort_after_crawl
from ollama_deep_researcher.translate_async import deduplicate_translate_and_format_sources
from langchain_groq import ChatGroq
from ollama_deep_researcher.state import SummaryState


In [ ]:

if __name__ == "__main__":
    search_r = await retsinfo_search("hovedforhandling formål afklaring parternes krav påstande forhandling", max_results=3, state=SummaryState)

In [ ]:
search_r

In [ ]:
if __name__ == "__main__":
    id_url_pari = await extract_ids(search_r)

In [ ]:
id_url_pari

In [ ]:
if __name__ == "__main__":
    crawl_r = await main_crawl(id_url_pari, "hovedforhandling formål afklaring parternes krav påstande forhandling")

In [ ]:
crawl_r[57411].error_message

In [ ]:
if __name__ == "__main__":
   sort_r = await sort_after_crawl(search_r, crawl_r)

In [ ]:
print(sort_r[221725])

In [ ]:

if __name__ == "__main__":
    # Load environment variables from .env file
   result = await retsinfo_search_and_crawl(query="hovedforhandling formål afklaring parternes krav påstande forhandling retssag", max_results=3, state=SummaryState)
    

In [ ]:
for r in result["results"]:
    print(r["ex_content"])
    for t in r:
        print(r["ex_content"])

In [ ]:
llm_translate_s = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=64000,
    service_tier="auto"
        )

In [ ]:
if __name__ == "__main__":
    search_str_en, saved_first_en = await deduplicate_translate_and_format_sources(result, llm_translate_s, 64000)

In [ ]:
print(search_str_en)

_____

# misc.

test qurry
```sh 
Lav en liste over hvad Justitsministeren kan fastsætte, I den juridiske contexten indsamlinger?
```

In [ ]:
array = [1,2]

array.append(3)

array[-2]

params = {"dt":[10,1480,20,30,40,50,90,120,270,60,100,80,110,130,140,150,160,170,180,200,210,220,1510,1490,-10,230,240,250,260,980]}

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import os
load_dotenv("../../.env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL")
base_url = os.getenv("OPENAI_BASE_URL")

In [ ]:
llm_translate_q = ChatOpenAI(
    model=model_name,
    base_url=base_url,
    temperature=1,
    max_tokens=12000,
)

In [ ]:
result = await llm_translate_q.ainvoke(
    [
        SystemMessage(content="you are a helpful assistant"),
        HumanMessage(content="what is the capital of France?"),
    ]
)

In [ ]:
text = """Lov om indsamling m.v.
VI MARGRETHE DEN ANDEN, af Guds Nåde Danmarks Dronning, gør vitterligt:
Folketinget har vedtaget og Vi ved Vort samtykke stadfæstet følgende lov:
Kapitel 1
Lovens anvendelsesområde
§ 1. Denne lov finder anvendelse på indsamlinger, hvorved forstås enhver form for opfordring til at yde bidrag til et forud angivet formål.
Stk. 2. § 8 og § 11, stk. 3, finder tilsvarende anvendelse for hjælpeorganisationers hvervning af medlemmer ved personlig eller telefonisk henvendelse.
Stk. 3. Salg anses som en indsamling, når det i umiddelbar tilknytning til salget tilkendegives, at en del af købesummen går til et forud angivet formål.
§ 2. Loven gælder ikke for følgende:
1) Private indsamlinger, hvorved forstås indsamlinger, hvor opfordringen rettes til indsamlerens bekendte eller til personer, som har særlig tilknytning til indsamlingsformålet.
2) Indsamlinger af brugte ting af ringe værdi.
3) Indsamlinger, der udelukkende finder sted ved indendørs gudstjenester eller møder, og som forestås af vedkommende trossamfund eller mødearrangør.
4) Indsamlinger foretaget blandt juridiske personer.
5) Indsamlinger til støtte for politiske partier.
6) Almennyttige lotterier.
Stk. 2. Uanset stk. 1, nr. 5, må indsamlinger til støtte for politiske partier ikke finde sted som hus- og gadeindsamlinger.
Kapitel 2
Anmeldelsespligt m.v.
§ 3. En indsamling skal anmeldes til Indsamlingsnævnet, inden indsamlingen iværksættes, jf. dog § 4, stk. 1. Ved anmeldelse af en indsamling skal formålet med indsamlingen anføres.
Stk. 2. Når indsamlingen er afsluttet, skal der indsendes regnskab herfor til Indsamlingsnævnet, jf. dog § 4, stk. 1.
§ 4. En organisation, der er godkendt af skatteministeren som berettiget til at modtage gaver, som kan fradrages ved indkomstopgørelsen, kan foretage indsamlinger uden anmeldelse og efterfølgende indsendelse af regnskab, hvis organisationen til Indsamlingsnævnet har indsendt oplysninger om
1) navn og adresse på organisationen,
2) på hvilken måde organisationen foretager indsamlinger, og
3) til hvilke formål indsamlede midler anvendes.
Stk. 2. Organisationen skal meddele Indsamlingsnævnet, hvis skatteministerens godkendelse nævnt i stk. 1 bortfalder, eller hvis forholdene omfattet af stk. 1, nr. 1-3, ændres.
Stk. 3. Organisationen skal indsende årsregnskab til Indsamlingsnævnet.
Stk. 4. Organisationen skal på en for offentligheden let tilgængelig måde oplyse, hvilke indsamlinger der er iværksat.
Kapitel 3
Hus- og gadeindsamling samt kædebreve
§ 5. Husindsamling må ikke finde sted, jf. dog stk. 2.
Stk. 2. Indsamlingsnævnet kan efter ansøgning meddele tilladelse for 3 år ad gangen til foretagelse af én årlig landsdækkende eller geografisk afgrænset husindsamling eller én enkeltstående husindsamling, som iværksættes på baggrund af en særlig begivenhed, hvis ansøgeren
1) er omfattet af § 4, stk. 1,
2) kan sandsynliggøre, at der ved indsamlingen indbringes et godt indsamlingsresultat, og
3) har den fornødne folkelige forankring.
Stk. 3. Tilladelse til husindsamling efter stk. 2 indebærer, at ansøgeren på dagen også kan foretage gadeindsamling.
§ 6. Gadeindsamling må ikke finde sted, jf. dog stk. 2 og § 5, stk. 3.
Stk. 2. Indsamlingsnævnet kan dog i særlige tilfælde efter ansøgning meddele tilladelse til foretagelse af gadeindsamling. Ansøgeren skal i forbindelse med ansøgningen opfylde betingelserne efter § 5, stk. 2, nr. 1-3.
§ 7. Indsamling ved anvendelse af kædebreve må ikke finde sted.
Kapitel 4
God indsamlingsskik
§ 8. Indsamlinger skal foregå i overensstemmelse med god indsamlingsskik.
Stk. 2. Klage over overtrædelse af stk. 1 skal indgives til Indsamlingsnævnet.
Kapitel 5
Indsamlingsnævnet
§ 9. Til varetagelse af opgaver efter denne lov oprettes der et indsamlingsnævn bestående af 1 formand og yderligere 4 medlemmer, som udpeges af justitsministeren. Formanden skal være jurist. 2 af de yderligere medlemmer beskikkes efter indstilling fra Indsamlingsorganisationernes Brancheorganisation, 1 medlem beskikkes efter indstilling fra Dansk Erhverv, og 1 medlem beskikkes efter indstilling fra Forbrugerrådet.
Stk. 2. Medlemmerne og stedfortrædere for disse, der udpeges efter samme regler, beskikkes for 4 år. Genbeskikkelse kan finde sted.
Stk. 3. Indsamlingsnævnets afgørelser kan ikke indbringes for anden administrativ myndighed.
§ 10. Indsamlingsnævnet fører tilsyn med, at indsamlinger foretages i overensstemmelse med denne lov, og fører en offentligt tilgængelig fortegnelse over anmeldte indsamlinger, jf. § 3, stk. 1, og de organisationer, der er omfattet af § 4, stk. 1, samt godkendte indsamlinger, jf. § 5, stk. 2, og § 6, stk. 2. Nævnet offentliggør regnskaber for indsamlinger på nævnets hjemmeside.
Stk. 2. Indsamlingsnævnet fører tilsyn med, at de indsamlede midler er anvendt i overensstemmelse med det formål, der er angivet i anmeldelsen, og at formålet er lovligt.
Stk. 3. Indsamlingsnævnet træffer afgørelse i sager om
1) tilladelse til foretagelse af husindsamlinger efter § 5, stk. 2, og gadeindsamlinger efter § 6, stk. 2,
2) tilladelse til anvendelse af indsamlede midler til andre formål end angivet i anmeldelsen og
3) overtrædelse af god indsamlingsskik, jf. § 8.
Stk. 4. Indsamlingsnævnet kan af egen drift optage en sag til behandling.
§ 11. Indsamlingsnævnet afgør, om en klage giver tilstrækkelig anledning til at iværksætte en nærmere undersøgelse.
Stk. 2. Hvis en klage ikke giver Indsamlingsnævnet anledning til kritik, kan sagen afsluttes, uden at Indsamlingsnævnet forinden forelægger klagen for den indklagede.
Stk. 3. Indsamlingsnævnet kan offentliggøre udtalelser af kritik samt pålægge en indsamler eller hjælpeorganisation at foretage offentliggørelse heraf.
§ 12. Indsamlingsnævnet kan hos en indsamler eller hjælpeorganisation kræve yderligere oplysninger til belysning af en anmeldelse, afgørelse eller klage eller af, om et forhold giver anledning til nærmere undersøgelse.
§ 13. Indsamlingsnævnet bistås af et sekretariat.
Stk. 2. Justitsministeren fastsætter Indsamlingsnævnets forretningsorden efter forhandling med nævnet.
Kapitel 6
Betaling og bemyndigelse
§ 14. Organisationer skal i forbindelse med indsamlinger omfattet af loven betale følgende:
1) 1.000 kr. i forbindelse med indgivelse af anmeldelse om indsamling.
2) 3.000 kr. årligt, hvis organisationen er omfattet af § 4, stk. 1.
3) 45.000 kr. i forbindelse med indgivelse af ansøgning om en 3-årig tilladelse til landsdækkende husindsamling, jf. § 5, stk. 2.
4) 22.500 kr. i forbindelse med indgivelse af ansøgning om regional husindsamling, jf. § 5, stk. 2.
5) 15.000 kr. i forbindelse med indgivelse af ansøgning om enkeltstående husindsamling, jf. § 5, stk. 2.
6) 6.000 kr. i forbindelse med indgivelse af ansøgning om landsdækkende gadeindsamling, jf. § 6, stk. 2.
7) 3.000 kr. i forbindelse med indgivelse af ansøgning om regional gadeindsamling, jf. § 6, stk. 2.
Stk. 2. Ansøgninger, jf. stk. 1, nr. 3-7, afvises, hvis betingelsen i stk. 1 ikke er opfyldt. Beløb efter stk. 1, nr. 3-7, tilbagebetales, hvis ansøgningen ikke imødekommes.
Stk. 3. De beløb, der er angivet i stk. 1, reguleres årligt pr. 1. januar med 2,0 pct. tillagt tilpasningsprocenten for det pågældende finansår, jf. lov om en satsreguleringsprocent. De herefter fremkomne beløb afrundes til nærmeste med 100 delelige kronebeløb. Reguleringen sker på grundlag af de på reguleringstidspunktet gældende beløb før afrunding.
Stk. 4. Indsamlingsnævnet offentliggør hvert år på nævnets hjemmeside de regulerede beløb, jf. stk. 3, for det kommende år.
Stk. 5. Justitsministeren fastsætter nærmere regler om betaling af de beløb, der er angivet i stk. 1, nr. 1-7, og om tilbagebetaling heraf.
§ 15. Justitsministeren fastsætter nærmere regler om fremgangsmåden ved indsamlinger og om den kontrol, som skal udøves med anvendelsen af de indsamlede midler. Justitsministeren fastsætter endvidere regler om, hvordan anmeldelse og indsendelse af regnskab skal ske. Justitsministeren fastsætter herudover nærmere regler om indhold og dokumentation i forbindelse med ansøgninger efter §§ 5 og 6.
Stk. 2. Justitsministeren kan endvidere fastsætte, at Indsamlingsnævnet med henblik på offentliggørelse skal afgive en årlig redegørelse om sin virksomhed til justitsministeren.
Kapitel 7
Straf og ikrafttræden m.v.
§ 16. Overtrædelse af § 2, stk. 2, § 3, stk. 1, § 5, stk. 1, § 6, stk. 1, og § 7 straffes med bøde, medmindre højere straf er forskyldt efter den øvrige lovgivning.
Stk. 2. I forskrifter, der udfærdiges i medfør af § 15, kan der fastsættes straf af bøde for overtrædelse af bestemmelserne i forskrifterne.
Stk. 3. Der kan pålægges selskaber m.v. (juridiske personer) strafansvar efter reglerne i straffelovens 5. kapitel.
§ 17. Loven træder i kraft den 1. juni 2014, jf. dog stk. 2.
Stk. 2. §§ 1-8, 10-12, 14 og 16 træder i kraft den 1. juli 2014.
Stk. 3. Lov om offentlige indsamlinger og pyramidespil, jf. lovbekendtgørelse nr. 555 af 6. juni 2006, ophæves den 1. juli 2014.
Stk. 4. Foreninger m.v., som inden den 1. juli 2014 har afholdt offentlig hus- og gadeindsamling i henhold til § 7, stk. 3 og 4, i lov om offentlige indsamlinger og pyramidespil, kan efter anmeldelse til Indsamlingsnævnet frem til den 1. juli 2017 foretage indsamling til samme formål og i samme form og udstrækning, hvortil tilladelse senest har været givet. De foreninger m.v., som vælger at benytte sig af muligheden for indsamling efter 1. pkt., vil ikke samtidig kunne meddeles tilladelse til husindsamling efter § 5, stk. 2. Anmeldelse efter 1. pkt. skal foretages senest den 1. januar 2015.
Stk. 5. Organisationer, som driver projektaktiviteter i udviklingslandene på grundlag af en rammeaftale med Udenrigsministeriet, og som inden den 1. juli 2014 har afholdt landsdækkende offentlig hus- og gadeindsamling i henhold til § 7, stk. 5, i lov om offentlige indsamlinger og pyramidespil, kan efter anmeldelse til Indsamlingsnævnet frem til den 1. juli 2017 foretage en årlig landsdækkende hus- og gadeindsamling til sådanne projektaktiviteter. De organisationer, som vælger at benytte sig af muligheden for indsamling efter 1. pkt., vil ikke samtidig kunne meddeles tilladelse til husindsamling efter § 5, stk. 2. Anmeldelse efter 1. pkt. skal foretages senest den 1. januar 2015.
§ 18. Loven gælder ikke for Færøerne og Grønland, men kan ved kongelig anordning sættes i kraft for Grønland med de ændringer, som de grønlandske forhold tilsiger.
Givet på Christiansborg Slot, den 26. maj 2014
Under Vor Kongelige Hånd og Segl
MARGRETHE R.
/ Karen Hækkerup"""

In [ ]:
class SlidingWindowChunking:
    def __init__(self, window_size=100, step=50):
        self.window_size = window_size
        self.step = step

    def chunk(self, text):
        words = text.split()
        chunks = []
        for i in range(0, len(words) - self.window_size + 1, self.step):
            chunks.append(' '.join(words[i:i + self.window_size]))
        return chunks


In [ ]:
from sentence_transformers import SentenceTransformer

class CosineSimilarityExtractor:
    def __init__(self, query):
        self.query = query
        self.model = SentenceTransformer("dilovancelik/all-distilroberta-v1_danish_law_fine_tune")

    def find_relevant_chunks(self, chunks):
        vectors = self.model.encode([self.query] + chunks)
        similarities = self.model.similarity(vectors[0:1], vectors[1:]).flatten()
        return [(chunks[i], similarities[i]) for i in range(len(chunks))]

# Example Workflow


chunker = SlidingWindowChunking(window_size=100, step=50)
chunks = chunker.chunk(text)
query = "Lav en liste over hvad Justitsministeren kan fastsætte, I den juridiske contexten indsamlinger?"
extractor = CosineSimilarityExtractor(query)
relevant_chunks = extractor.find_relevant_chunks(chunks)
final = ""
for r,s in relevant_chunks:
    if s > 0.2:
        final += f"{r}\n"

In [ ]:
print(final)